## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 23rd Mar. 2nd


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-28-2025,Condo/Co-op,1776 Polk St #705,Hollywood,FL,33020.0,405175.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hollywood/1776-Polk-...,MARMLS,A11754645,N,Y,26.013097,-80.142217


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-28    39
2025-02-26    33
2025-02-25    31
2025-02-24    30
2025-02-27    17
2025-03-01     2
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
64,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777
123,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10456723,N,Y,26.132895,-80.104208
65,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Beaches MLS,RX-11046017,N,Y,26.293859,-80.078549
72,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Beaches MLS,F10476288,N,Y,26.131573,-80.103739
148,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Beaches MLS,F10429275,N,Y,26.121273,-80.105256
51,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,MARMLS,A11703957,N,Y,26.226042,-80.090808
136,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10446337,N,Y,26.095913,-80.106280
138,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11595527,N,Y,25.984216,-80.118868
7,PAST SALE,2025-02-25,Condo/Co-op,4100 Galt Ocean Dr #1108,Fort Lauderdale,FL,33308.0,850000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/4100...,MARMLS,A11731771,N,Y,26.178986,-80.097056
88,PAST SALE,2025-02-27,Condo/Co-op,511 Bayshore Dr Ph 10,Fort Lauderdale,FL,33304.0,825000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/511-...,MARMLS,A11697806,N,Y,26.129150,-80.108666


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,...,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None
1,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,...,-80.104208,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Reinaldo Cunha,Alrei Real Estate Services LLC,Hans De Beer,Alrei Real Estate Services LLC,Joseph McFarlane,Reform Realty,Joseph McFarlane,Reform Realty
2,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,...,-80.078549,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Robert Meyer,Keller Williams Realty Services,None,None,Robert Meyer,Keller Williams Realty Services,None,None
3,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,...,-80.103739,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Donald Ciarlillo,Atlantic Luxury Properties Inc,None,None,Richard Bannon P.A.,One Sotheby's Int'l Realty,None,None
4,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,...,-80.105256,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None
5,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,...,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Laurie Spector,"Compass Florida, LLC",None,None,None,None,None,None
6,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,...,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Martha Buckley,Coldwell Banker Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None
7,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,...,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,None,None,None,None,None,None,None,None
8,PAST SALE,2025-02-25,Condo/Co-op,4100 Galt Ocean Dr #1108,Fort Lauderdale,FL,33308.0,850000.0,2.0,2.0,...,-80.097056,https://www.redfin.com/FL/Fort-Lauderdale/4100...,Richard Bonvie,Bonvie Realty,None,None,None,None,None,None
9,PAST SALE,2025-02-27,Condo/Co-op,511 Bayshore Dr Ph 10,Fort Lauderdale,FL,33304.0,825000.0,2.0,2.0,...,-80.108666,https://www.redfin.com/FL/Fort-Lauderdale/511-...,Barbara Veitch,Galleria International Realty,None,None,None,None,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

152


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$374,763


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$295


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$56,964,015


In [24]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PDF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PDF  Last Week Total Dollar Volume
0          152              374763          295                     56964015.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [25]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Broward County
Input Previous Week Condo Sales Total (number units sold):
101
Input Previous Week Condo Average Sales Price:
322632
Input Previous Week Condo Average PSF:
269
Input Previous Week Condo Sales total (ex: 198_000_000)
32_600_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [26]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [27]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [28]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [29]:
pd.set_option('display.max_columns',None)

In [30]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange


In [31]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [32]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange
1,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,Paramount Residences,2420.0,NaN,2019.0,NaN,1136.0,4092.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10456723,N,Y,26.132895,-80.104208,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Reinaldo Cunha,Alrei Real Estate Services LLC,Hans De Beer,Alrei Real Estate Services LLC,Joseph McFarlane,Reform Realty,Joseph McFarlane,Reform Realty,2,blue
2,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,Hillsboro Ocean Club Condo,2500.0,NaN,1984.0,NaN,860.0,1923.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Beaches MLS,RX-11046017,N,Y,26.293859,-80.078549,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Robert Meyer,Keller Williams Realty Services,None,None,Robert Meyer,Keller Williams Realty Services,None,None,3,blue
3,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,The Atlantic Hotel Condo,2450.0,NaN,2004.0,NaN,765.0,113.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Beaches MLS,F10476288,N,Y,26.131573,-80.103739,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Donald Ciarlillo,Atlantic Luxury Properties Inc,None,None,Richard Bannon P.A.,One Sotheby's Int'l Realty,None,None,4,blue
4,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,Las Olas Beach Club,2380.0,NaN,2006.0,NaN,666.0,3010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Beaches MLS,F10429275,N,Y,26.121273,-80.105256,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,5,blue
5,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,SONATA BEACH CLUB CONDO,2561.0,NaN,2008.0,NaN,489.0,2963.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,MARMLS,A11703957,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Laurie Spector,"Compass Florida, LLC",None,None,None,None,None,None,6,blue
6,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,Point Of Americas,2170.0,NaN,1971.0,NaN,495.0,2192.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10446337,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Martha Buckley,Coldwell Banker Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,7,blue
7,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,BEACH CLUB CONDO,1571.0,NaN,2005.0,NaN,560.0,1616.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hallandale-Beach/185...,MARMLS,A11595527,N,Y,25.984216,-80.118868,https://www.redfin.com/FL/Hallandale-Beach/185...,None,None,None,None,None,None,None,None,8,blue
8,PAST SALE,2025-02-25,Condo/Co-op,4100 Galt Ocean Dr #1108,Fort Lauderdale,FL,33308.0,850000.0,2.0,2.0,GALLEON CONDO

In [33]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [34]:
m.save('index.html')

## Data snagger

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [36]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [37]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Broward_County_condo_sales_week_ending_03032024


# CREATE TEMPLATE 

In [38]:
muni_set = set(df_top_ten['CITY'])

In [39]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [40]:
df_top_ten.reset_index(inplace=True,drop=True)

In [41]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [42]:
top_sale

'Solemar at 1116 N Ocean Blvd #701 in Pompano Beach'

In [43]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [44]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [45]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange,Solemar at 1116 N Ocean Blvd #701 in Pompano B...
1,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,Paramount Residences,2420.0,NaN,2019.0,NaN,1136.0,4092.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10456723,N,Y,26.132895,-80.104208,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Reinaldo Cunha,Alrei Real Estate Services LLC,Hans De Beer,Alrei Real Estate Services LLC,Joseph McFarlane,Reform Realty,Joseph McFarlane,Reform Realty,2,blue,Paramount Residences at 701 N Fort Lauderdale ...
2,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,Hillsboro Ocean Club Condo,2500.0,NaN,1984.0,NaN,860.0,1923.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Beaches MLS,RX-11046017,N,Y,26.293859,-80.078549,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Robert Meyer,Keller Williams Realty Services,None,None,Robert Meyer,Keller Williams Realty Services,None,None,3,blue,Hillsboro Ocean Club Condo at 1155 Hillsboro M...
3,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,The Atlantic Hotel Condo,2450.0,NaN,2004.0,NaN,765.0,113.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Beaches MLS,F10476288,N,Y,26.131573,-80.103739,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Donald Ciarlillo,Atlantic Luxury Properties Inc,None,None,Richard Bannon P.A.,One Sotheby's Int'l Realty,None,None,4,blue,The Atlantic Hotel Condo at 601 N Ft Lauderdal...
4,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,Las Olas Beach Club,2380.0,NaN,2006.0,NaN,666.0,3010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Beaches MLS,F10429275,N,Y,26.121273,-80.105256,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,5,blue,Las Olas Beach Club at 101 S Fort Lauderdale B...
5,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,SONATA BEACH CLUB CONDO,2561.0,NaN,2008.0,NaN,489.0,2963.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,MARMLS,A11703957,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Laurie Spector,"Compass Florida, LLC",None,None,None,None,None,None,6,blue,SONATA BEACH CLUB CONDO at 801 Briny Ave #605 ...
6,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,Point Of Americas,2170.0,NaN,1971.0,NaN,495.0,2192.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10446337,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Martha Buckley,Coldwell Banker Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,7,blue,Point Of Americas at 2200 S Ocean Ln #905 in F...
7,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,BEACH CLUB 

In [46]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Solemar closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $825,000 to $3,450,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Pompano Beach, Fort Lauderdale, Hallandale Beach, Hillsboro Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 152 condo sales totaling $56,964,015 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 101 condo sales totaling $32,600,000.

Last week’s units sold for an average of $374,763, higher than the $322,632

In [47]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [48]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange,Solemar at 1116 N Ocean Blvd #701 in Pompano B...
1,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,Paramount Residences,2420.0,NaN,2019.0,NaN,1136.0,4092.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10456723,N,Y,26.132895,-80.104208,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Reinaldo Cunha,Alrei Real Estate Services LLC,Hans De Beer,Alrei Real Estate Services LLC,Joseph McFarlane,Reform Realty,Joseph McFarlane,Reform Realty,2,blue,Paramount Residences at 701 N Fort Lauderdale ...
2,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,Hillsboro Ocean Club Condo,2500.0,NaN,1984.0,NaN,860.0,1923.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Beaches MLS,RX-11046017,N,Y,26.293859,-80.078549,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Robert Meyer,Keller Williams Realty Services,None,None,Robert Meyer,Keller Williams Realty Services,None,None,3,blue,Hillsboro Ocean Club Condo at 1155 Hillsboro M...
3,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,The Atlantic Hotel Condo,2450.0,NaN,2004.0,NaN,765.0,113.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Beaches MLS,F10476288,N,Y,26.131573,-80.103739,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Donald Ciarlillo,Atlantic Luxury Properties Inc,None,None,Richard Bannon P.A.,One Sotheby's Int'l Realty,None,None,4,blue,The Atlantic Hotel Condo at 601 N Ft Lauderdal...
4,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,Las Olas Beach Club,2380.0,NaN,2006.0,NaN,666.0,3010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Beaches MLS,F10429275,N,Y,26.121273,-80.105256,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,5,blue,Las Olas Beach Club at 101 S Fort Lauderdale B...
5,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,SONATA BEACH CLUB CONDO,2561.0,NaN,2008.0,NaN,489.0,2963.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,MARMLS,A11703957,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Laurie Spector,"Compass Florida, LLC",None,None,None,None,None,None,6,blue,SONATA BEACH CLUB CONDO at 801 Briny Ave #605 ...
6,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,Point Of Americas,2170.0,NaN,1971.0,NaN,495.0,2192.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10446337,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Martha Buckley,Coldwell Banker Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,7,blue,Point Of Americas at 2200 S Ocean Ln #905 in F...
7,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,BEACH CLUB 

In [51]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Solemar closing tops Broward County weekly condo sales 
DEK: Top 10 sales ranged from $825,000 to $3,450,000
FEATURED HED:
SEO HED: Broward County County Weekly Condo Report 
SEO DESCRIPTION: Broward County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Broward County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Broward County, weekly condo sales, Pompano Beach, Fort Lauderdale, Hallandale Beach, Hillsboro Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Broward County County’s NEWS PEG HERE.

Brokers closed 152 condo sales totaling $56,964,015 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 101 condo sales totaling $32,600,000.

Last week’s units sold for an average of $374,763, higher than the $322,632

In [52]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-701/home/190140467


In [60]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Fort-Lauderdale/701-N-Fort-Lauderdale-Beach-Blvd-33304/unit-1206/home/145072139


In [54]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Fort-Lauderdale/511-Bayshore-Dr-33304/unit-10/home/41782953


In [55]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Fort-Lauderdale/4100-Galt-Ocean-Dr-33308/unit-1108/home/41707681


In [56]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Pompano-Beach/1116-N-Ocean-Blvd-33062/unit-701/home/190140467


In [57]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Fort-Lauderdale/4100-Galt-Ocean-Dr-33308/unit-1108/home/41707681


In [58]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-27,Condo/Co-op,1116 N Ocean Blvd #701,Pompano Beach,FL,33062.0,3450000.0,3.0,3.5,Solemar,2825.0,NaN,2023.0,NaN,1221.0,3769.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/1116-N...,Beaches MLS,F10471866,N,Y,26.247536,-80.086777,https://www.redfin.com/FL/Pompano-Beach/1116-N...,David Marder,"Compass Florida, LLC",None,None,Marisa DiLenge,Realty 100,None,None,1,orange,Solemar at 1116 N Ocean Blvd #701 in Pompano B...
1,PAST SALE,2025-02-26,Condo/Co-op,701 N Fort Lauderdale Beach Blvd #1206,Fort Lauderdale,FL,33304.0,2750000.0,3.0,3.5,Paramount Residences,2420.0,NaN,2019.0,NaN,1136.0,4092.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Beaches MLS,F10456723,N,Y,26.132895,-80.104208,https://www.redfin.com/FL/Fort-Lauderdale/701-...,Reinaldo Cunha,Alrei Real Estate Services LLC,Hans De Beer,Alrei Real Estate Services LLC,Joseph McFarlane,Reform Realty,Joseph McFarlane,Reform Realty,2,blue,Paramount Residences at 701 N Fort Lauderdale ...
2,PAST SALE,2025-02-28,Condo/Co-op,1155 Hillsboro Mile #108,Hillsboro Beach,FL,33062.0,2150000.0,3.0,2.5,Hillsboro Ocean Club Condo,2500.0,NaN,1984.0,NaN,860.0,1923.0,Sold,NaN,NaN,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Beaches MLS,RX-11046017,N,Y,26.293859,-80.078549,https://www.redfin.com/FL/Hillsboro-Beach/1155...,Robert Meyer,Keller Williams Realty Services,None,None,Robert Meyer,Keller Williams Realty Services,None,None,3,blue,Hillsboro Ocean Club Condo at 1155 Hillsboro M...
3,PAST SALE,2025-02-26,Condo/Co-op,601 N Ft Lauderdale Bch Blvd #1502,Fort Lauderdale,FL,33304.0,1875000.0,2.0,2.5,The Atlantic Hotel Condo,2450.0,NaN,2004.0,NaN,765.0,113.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Beaches MLS,F10476288,N,Y,26.131573,-80.103739,https://www.redfin.com/FL/Fort-Lauderdale/601-...,Donald Ciarlillo,Atlantic Luxury Properties Inc,None,None,Richard Bannon P.A.,One Sotheby's Int'l Realty,None,None,4,blue,The Atlantic Hotel Condo at 601 N Ft Lauderdal...
4,PAST SALE,2025-02-28,Condo/Co-op,101 S Fort Lauderdale Beach Blvd #902,Fort Lauderdale,FL,33316.0,1584000.0,3.0,3.0,Las Olas Beach Club,2380.0,NaN,2006.0,NaN,666.0,3010.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Beaches MLS,F10429275,N,Y,26.121273,-80.105256,https://www.redfin.com/FL/Fort-Lauderdale/101-...,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,Tyler Tuchow,Fortune Christie's International Real Estate,None,None,5,blue,Las Olas Beach Club at 101 S Fort Lauderdale B...
5,PAST SALE,2025-02-27,Condo/Co-op,801 Briny Ave #605,Pompano Beach,FL,33062.0,1252000.0,2.0,2.5,SONATA BEACH CLUB CONDO,2561.0,NaN,2008.0,NaN,489.0,2963.0,Sold,NaN,NaN,https://www.redfin.com/FL/Pompano-Beach/801-Br...,MARMLS,A11703957,N,Y,26.226042,-80.090808,https://www.redfin.com/FL/Pompano-Beach/801-Br...,Laurie Spector,"Compass Florida, LLC",None,None,None,None,None,None,6,blue,SONATA BEACH CLUB CONDO at 801 Briny Ave #605 ...
6,PAST SALE,2025-02-28,Condo/Co-op,2200 S Ocean Ln #905,Fort Lauderdale,FL,33316.0,1075000.0,2.0,2.0,Point Of Americas,2170.0,NaN,1971.0,NaN,495.0,2192.0,Sold,NaN,NaN,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Beaches MLS,F10446337,N,Y,26.095913,-80.106280,https://www.redfin.com/FL/Fort-Lauderdale/2200...,Martha Buckley,Coldwell Banker Realty,None,None,Brian Hero,One Sotheby's Int'l Realty,None,None,7,blue,Point Of Americas at 2200 S Ocean Ln #905 in F...
7,PAST SALE,2025-02-28,Condo/Co-op,1850 S Ocean Dr #1207,Hallandale Beach,FL,33009.0,880000.0,3.0,3.0,BEACH CLUB 

## Time on Market Calculator

In [61]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 8, 16) ## List (Earlier) date
date2 = datetime(2025, 2, 26) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

194


## Clean CSV for Datawrapper Chart

In [62]:
chart_df = df_top_ten

In [63]:
chart_df = chart_df.fillna(" ")

In [64]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [65]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [66]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [67]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [68]:
chart_df['$/SQUARE FEET'].astype(float)

0    1221.0
1    1136.0
2     860.0
3     765.0
4     666.0
5     489.0
6     495.0
7     560.0
8     438.0
9     620.0
Name: $/SQUARE FEET, dtype: float64

In [69]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [70]:
csv_date_string = today.strftime("%m_%d_%Y")

In [71]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [72]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-27,1116 N Ocean Blvd #701 in Pompano Beach,"$3,450,000",3,3.5,"2,825",2023,"$1,221",Beaches MLS,F10471866,"David Marder Compass Florida, LLC",Marisa DiLenge Realty 100
1,2025-02-26,701 N Fort Lauderdale Beach Blvd #1206 in Fort...,"$2,750,000",3,3.5,"2,420",2019,"$1,136",Beaches MLS,F10456723,Reinaldo Cunha Alrei Real Estate Services LLC ...,Joseph McFarlane Reform Realty Joseph McFarlan...
2,2025-02-28,1155 Hillsboro Mile #108 in Hillsboro Beach,"$2,150,000",3,2.5,"2,500",1984,$860,Beaches MLS,RX-11046017,Robert Meyer Keller Williams Realty Services,Robert Meyer Keller Williams Realty Services
3,2025-02-26,601 N Ft Lauderdale Bch Blvd #1502 in Fort Lau...,"$1,875,000",2,2.5,"2,450",2004,$765,Beaches MLS,F10476288,Donald Ciarlillo Atlantic Luxury Properties In...,Richard Bannon P.A. One Sotheby's Int'l Realty...
4,2025-02-28,101 S Fort Lauderdale Beach Blvd #902 in Fort ...,"$1,584,000",3,3.0,"2,380",2006,$666,Beaches MLS,F10429275,Tyler Tuchow Fortune Christie's International ...,Tyler Tuchow Fortune Christie's International ...
5,2025-02-27,801 Briny Ave #605 in Pompano Beach,"$1,252,000",2,2.5,"2,561",2008,$489,MARMLS,A11703957,"Laurie Spector Compass Florida, LLC",
6,2025-02-28,2200 S Ocean Ln #905 in Fort Lauderdale,"$1,075,000",2,2.0,"2,170",1971,$495,Beaches MLS,F10446337,Martha Buckley Coldwell Banker Realty,Brian Hero One Sotheby's Int'l Realty
7,2025-02-28,1850 S Ocean Dr #1207 in Hallandale Beach,"$880,000",3,3.0,"1,571",2005,$560,MARMLS,A11595527,,
8,2025-02-25,4100 Galt Ocean Dr #1108 in Fort Lauderdale,"$850,000",2,2.0,"1,940",1967,$438,MARMLS,A11731771,Richard Bonvie Bonvie Realty,
9,2025-02-27,511 Bayshore Dr Ph 10 in Fort Lauderdale,"$825,000",2,2.0,"1,330",1971,$620,MARMLS,A11697806,Barbara Veitch Galleria International Realty,
